# Lab: Cassandra CRUD Operations

![https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-WD0231EN-SkillsNetwork/IDSN-logo.png](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-WD0231EN-SkillsNetwork/IDSN-logo.png)

Estimated time needed: **15** minutes

## **Objectives**

After completing this lab, you will be able to:

- Insert data into a table with an INSERT command
- Read data from a table by querying from it
- Update and delete data from the table based on specific criteria

## Open Docker



In [1]:
!open /Applications/Docker.app

In [2]:
!docker version

Client:
 Cloud integration: v1.0.35+desktop.5
 Version:           24.0.6
 API version:       1.43
 Go version:        go1.20.7
 Git commit:        ed223bc
 Built:             Mon Sep  4 12:28:49 2023
 OS/Arch:           darwin/arm64
 Context:           desktop-linux

Server: Docker Desktop 4.24.0 (122432)
 Engine:
  Version:          24.0.6
  API version:      1.43 (minimum version 1.12)
  Go version:       go1.20.7
  Git commit:       1a79695
  Built:            Mon Sep  4 12:31:36 2023
  OS/Arch:          linux/arm64
  Experimental:     false
 containerd:
  Version:          1.6.22
  GitCommit:        8165feabfdfe38c65b599c4993d227328c231fca
 runc:
  Version:          1.1.8
  GitCommit:        v1.1.8-0-g82f18fe
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0




## **Deploying your Cassandra cluster and running commands**

To deploy your Cassandra cluster, use the Docker CLI in the same folder as your docker-compose.yml to run the following command (the -d causes the containers to run in the background):



In [3]:
!docker compose up -d

[+] Running 0/0
 ⠙ cassandra1 Pulling                                                      0.1s 
 ⠙ cassandra2 Pulling                                                      0.1s 
 ⠙ cassandra3 Pulling                                                      0.1s 
[+] Running 0/3
 ⠹ cassandra1 Pulling                                                      0.2s 
 ⠹ cassandra2 Pulling                                                      0.2s 
 ⠹ cassandra3 Pulling                                                      0.2s 
[+] Running 0/3
 ⠸ cassandra1 Pulling                                                      0.3s 
 ⠸ cassandra2 Pulling                                                      0.3s 
 ⠸ cassandra3 Pulling                                                      0.3s 
[+] Running 0/3
 ⠼ cassandra1 Pulling                                                      0.4s 
 ⠼ cassandra2 Pulling                                                      0.4s 
 ⠼ cassandra3 Pulling                        



To access your Cassandra cluster, you can use csqlsh to connect to the container database using the following commands:





You can also check the cluster configuration using:



In [9]:
!docker exec -it cassandra1 nodetool status

Datacenter: DC1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UJ  172.18.0.4  182.62 KiB  128     ?                 fc8047e1-17a2-4077-bcb3-cdf15f9cf798  RACK1
UN  172.18.0.3  72.44 KiB   128     100.0%            83bc7599-b9ba-4a39-b652-31c68c7f3c34  RACK1
UN  172.18.0.2  72.41 KiB   128     100.0%            061d22d2-d610-4dc3-95bb-cad8f5cdf57b  RACK1



In [10]:
!docker ps

CONTAINER ID   IMAGE              COMMAND                  CREATED         STATUS         PORTS                                                       NAMES
9f88222260c7   cassandra:latest   "docker-entrypoint.s…"   2 minutes ago   Up 2 minutes   7000-7001/tcp, 7199/tcp, 9160/tcp, 0.0.0.0:9044->9042/tcp   cassandra3
40c9bdc93c4b   cassandra:latest   "docker-entrypoint.s…"   2 minutes ago   Up 2 minutes   7000-7001/tcp, 7199/tcp, 9160/tcp, 0.0.0.0:9043->9042/tcp   cassandra2
b7c52c54342f   cassandra:latest   "docker-entrypoint.s…"   2 minutes ago   Up 2 minutes   7000-7001/tcp, 7199/tcp, 9160/tcp, 0.0.0.0:9042->9042/tcp   cassandra1




# **Exercise 1 - Create Keyspace and Table**

## **Create `training` keyspace**

Create a keyspace named `training` using `SimpleStrategy` and `replication factor` of `3`.

- Click here for Hint

> use CREATE KEYSPACEcommand with appropriate options
>
- Click here for Solution

On the cqlsh run the below command.



In [11]:
!docker exec -it cassandra1 cqlsh -e "CREATE KEYSPACE training WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 2};"

## **Install `Python` Client**

In [12]:
%pip install cassandra-driver

  Obtaining dependency information for cassandra-driver from https://files.pythonhosted.org/packages/f5/75/c9e6be97ec924943553cbdf03f0de39aa128aafb95f715d780a1784d0c81/cassandra_driver-3.29.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for geomet<0.3,>=0.1 from https://files.pythonhosted.org/packages/c9/81/156ca48f950f833ddc392f8e3677ca50a18cb9d5db38ccb4ecea55a9303f/geomet-0.2.1.post1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 3.8 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


## **Connect to Cassandra**
Below is a Python script that connects to the Cassandra cluster and selects the training keyspace

In [14]:
from cassandra.cluster import Cluster

# Connection to the Cassandra node (host's IP and mapped port)
cassandra_host = '127.0.0.1'
cassandra_port = 9042

# Connect to the Cassandra cluster
cluster = Cluster([cassandra_host], port=cassandra_port)
session = cluster.connect()

# Select the keyspace
session.set_keyspace('training')




## **Create `movies` table**

Create a table named `movies` with columns:

- `movie_id` is an integer and is the primary key.
- `movie_name` is a text column.
- `year_of_release` is an integer.
- Click here for Hint

> use CREATE TABLE command with appropriate options
>
- Click here for Solution

On the cqlsh run the below command.



In [15]:
# Create the 'movies' table
query = '''
CREATE TABLE IF NOT EXISTS movies (
    movie_id int PRIMARY KEY,
    movie_name text,
    year_of_release int
)
'''
session.execute(query)




# **Exercise 2 - Insert data into a table**

Let's insert a row into the table movies.

On cqlsh run the below command.



In [16]:
query = '''
INSERT into movies(
movie_id, movie_name, year_of_release)
VALUES (1,'Toy Story',1995);
'''
session.execute(query)



Verify that the data is saved.



In [17]:
query = '''
select * from movies;
'''
session.execute(query)

In [19]:
import pandas as pd

# Execute a query
query = '''
SELECT * FROM movies;
'''
result_set = session.execute(query)

# Convert ResultSet to a list of dictionaries
rows = [dict(row._asdict()) for row in result_set]

# Create a DataFrame
df = pd.DataFrame(rows)

# Display the DataFrame
df

,movie_id,movie_name,year_of_release
0,1,Toy Story,1995




Insert the below movies into the table.

| movie_id | movie_name | year_of_release |
| --- | --- | --- |
| 2 | Jumanji | 1995 |
| 3 | Heat | 1995 |
| 4 | Scream | 1995 |
| 5 | Fargo | 1996 |



In [21]:
# Insert records into the 'movies' table
insert_queries = [
    "INSERT INTO movies (movie_id, movie_name, year_of_release) VALUES (2, 'Jumanji', 1995);",
    "INSERT INTO movies (movie_id, movie_name, year_of_release) VALUES (3, 'Heat', 1995);",
    "INSERT INTO movies (movie_id, movie_name, year_of_release) VALUES (4, 'Scream', 1995);",
    "INSERT INTO movies (movie_id, movie_name, year_of_release) VALUES (5, 'Fargo', 1996);"
]

for query in insert_queries:
    session.execute(query)



# **Exercise 3 - Read data from a table**

In the previous exercise you have inserted some data into the table named `movies`.

Let's query the data in the `movies` table.

Query all rows.



In [23]:
# Execute a query to retrieve the records
query = '''
SELECT * FROM movies;
'''
result_set = session.execute(query)

# Convert ResultSet to a list of dictionaries
rows = [dict(row._asdict()) for row in result_set]

# Create a DataFrame
df = pd.DataFrame(rows)

# Display the DataFrame
df

,movie_id,movie_name,year_of_release
0,5,Fargo,1996
1,1,Toy Story,1995
2,2,Jumanji,1995
3,4,Scream,1995
4,3,Heat,1995


In [25]:
# Execute a query to retrieve the records without ordering
query = '''
SELECT * FROM movies;
'''
result_set = session.execute(query)

# Convert ResultSet to a list of dictionaries
rows = [dict(row._asdict()) for row in result_set]

# Create a DataFrame
df = pd.DataFrame(rows)

# Sort the DataFrame by movie_id
df_sorted = df.sort_values(by='movie_id')

# Display the sorted DataFrame
df_sorted

,movie_id,movie_name,year_of_release
1,1,Toy Story,1995
2,2,Jumanji,1995
4,3,Heat,1995
3,4,Scream,1995
0,5,Fargo,1996




Query the movie name where `movie_id` is `1`.



In [26]:
# Query to retrieve the movie name where movie_id is 1
query = '''
SELECT movie_name FROM movies WHERE movie_id = 1;
'''
result_set = session.execute(query)

# Convert ResultSet to a list of dictionaries
rows = [dict(row._asdict()) for row in result_set]

# Create a DataFrame
df = pd.DataFrame(rows)

# Display the DataFrame
df


,movie_name
0,Toy Story




# **Exercise 4 - Update data in a table**

The `movie_id` for Scream is `4`. It was released in 1996 and not 1995.

Here is how you modify it.



In [27]:
# Update the year_of_release for movie_id 4
update_query = '''
UPDATE movies
SET year_of_release = 1996
WHERE movie_id = 4;
'''
session.execute(update_query)



Verify that the update was successful.



In [28]:
query = '''
select * from movies where movie_id = 4;
'''
result_set = session.execute(query)

# Convert ResultSet to a list of dictionaries
rows = [dict(row._asdict()) for row in result_set]

# Create a DataFrame
df = pd.DataFrame(rows)

# Display the DataFrame
df

,movie_id,movie_name,year_of_release
0,4,Scream,1996




# **Exercise 5 - Delete data from a table**

Delete the movie with the movie_id 5.



In [29]:
# Delete the record with movie_id 5
delete_query = '''
DELETE FROM movies
WHERE movie_id = 5;
'''
session.execute(delete_query)



Verify using the below command.



In [31]:
query = '''
select * from movies;
'''
result_set = session.execute(query)

# Convert ResultSet to a list of dictionaries
rows = [dict(row._asdict()) for row in result_set]

# Sort the DataFrame by movie_id
df_sorted = df.sort_values(by='movie_id')

# Display the sorted DataFrame
df_sorted

,movie_id,movie_name,year_of_release
0,1,Toy Story,1995
1,2,Jumanji,1995
3,3,Heat,1995
2,4,Scream,1996




# **Practice exercises**

1. Problem: Insert the below movie into the movies table.


| movie_id | movie_name | year_of_release |
| --- | --- | --- |
| 6 | Twister | 1997 |

> use the command insert into table’
>





1. Problem: Modify the release year of Twister to 1996.

> use the command ‘update table set column=value where column=value’
>

On the cqlsh run the below command.





1. Problem: Delete the movie Twister.

> use the command delete from ...
>

On the cqlsh run the below command.





1. Problem: Drop the `training` keyspace.

> use the drop keyspace command
>

On the cqlsh run the below command.





# **Summary**

In this lab, you have gained an understanding of CRUD operations in Cassandra.

In [32]:
# Close the connection
cluster.shutdown()